# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
# try this project here!

In [1]:
import torch as th
from torch import nn, optim
import syft as sy

hook = sy.TorchHook(th)

In [2]:
user1 = sy.VirtualWorker(hook, id='user1')
user2 = sy.VirtualWorker(hook, id='user2')
concentrator = sy.VirtualWorker(hook, id='concentrator')

user1.add_workers([user2, concentrator])
user2.add_workers([user1, concentrator])
concentrator.add_workers([user1, user2])

print(user1._objects)
print(user2._objects)
print(concentrator._objects)

{}
{}
{}


In [3]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

# Distribute the datsets
data1 = data[:2].send(user1)
target1 = target[:2].send(user1)

data2 = data[2:].send(user2)
target2 = target[2:].send(user2)

datasets = [(data1, target1, user1), (data2, target2, user2)]

In [16]:
def train(iterations=20, steps_per_round=10):

    model = nn.Linear(2,1)
    
    for round_iter in range(iterations):
        # Distribute the models
        user1_model = model.copy().send(user1)
        user1_opt = optim.SGD(params=user1_model.parameters(), lr=0.1)
        user2_model = model.copy().send(user2)
        user2_opt = optim.SGD(params=user2_model.parameters(), lr=0.1)

        for i in range(steps_per_round):
            # user1 training
            user1_opt.zero_grad()
            user1_pred = user1_model(data1)
            user1_loss = ((user1_pred - target1)**2).sum()
            user1_loss.backward()
            user1_opt.step()

            # user2 training
            user2_opt.zero_grad()
            user2_pred = user2_model(data2)
            user2_loss = ((user2_pred - target2)**2).sum()
            user2_loss.backward()
            user2_opt.step()

        # Aggregate the models
        user1_model.move(concentrator)
        user2_model.move(concentrator)
        with th.no_grad():
            model.weight.set_(((user1_model.weight.data + user2_model.weight.data) / 2).get())
            model.bias.set_(((user1_model.bias.data + user2_model.bias.data) / 2).get())
        
        # Print the loss
        user1_loss = user1_loss.get()
        user2_loss = user2_loss.get()
        print('user1 loss: {},   user2 loss: {}'.format(user1_loss, user2_loss))
    
    concentrator.clear_objects()

    return model

In [17]:
model = train(50, 10)

user1 loss: 0.013930113986134529,   user2 loss: 0.010047199204564095
user1 loss: 0.0009704395197331905,   user2 loss: 0.001761196763254702
user1 loss: 9.99339681584388e-05,   user2 loss: 0.0004806247306987643
user1 loss: 7.5853258749702945e-06,   user2 loss: 0.00017417926574125886
user1 loss: 3.2725786525134026e-09,   user2 loss: 7.971627928782254e-05
user1 loss: 1.2485622846725164e-06,   user2 loss: 4.39705399912782e-05
user1 loss: 2.0801594473596197e-06,   user2 loss: 2.7636400773189962e-05
user1 loss: 2.135548129444942e-06,   user2 loss: 1.883457116491627e-05
user1 loss: 1.8529929093347164e-06,   user2 loss: 1.3443372154142708e-05
user1 loss: 1.4944647546144552e-06,   user2 loss: 9.839644917519763e-06
user1 loss: 1.164136961051554e-06,   user2 loss: 7.298437594727147e-06
user1 loss: 8.915098987927195e-07,   user2 loss: 5.451198376249522e-06
user1 loss: 6.767943432350876e-07,   user2 loss: 4.086145054316148e-06
user1 loss: 5.116820034345437e-07,   user2 loss: 3.068585328946938e-06
us

In [18]:
model(data)

tensor([[9.9993e-01],
        [9.9994e-01],
        [4.6894e-05],
        [5.0795e-05]], grad_fn=<AddmmBackward>)

In [19]:
target

tensor([[1.],
        [1.],
        [0.],
        [0.]], requires_grad=True)

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [20]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [21]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [15]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [22]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [23]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [24]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [25]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [32]:
# try this project here!
import numpy as np

In [67]:
def encrypt(x, n_splits, Q):
    """
    Args:
        x(int): The number to 'encrypt'
        n_splits(int): The number of workers to split the shares.
        Q(int): The large number to use for the modulus operation
    Returns:
        tuple: The tuple of shares
    """
    shares = np.random.randint(0, Q, size=n_splits)
    shares[-1] = Q - (shares[:-1].sum() % Q) + x
    return tuple(shares)

In [70]:
def decrypt(shares, Q):
    """
    Args:
        shares(tuple): The 'encrypted' shares
    Returns:
        int: The decrypted value
    """
    return sum(shares) % Q

In [71]:
def add(share1, share2):
    return (s1 + s2 for s1, s2 in zip(share1, share2))

In [72]:
Q = 23740629843760
shares = encrypt(675, 2, Q)
shares

(18102742910283, 5637886934152)

In [73]:
decrypt(shares, Q)

675

In [74]:
a = 342
b = 765
a + b

1107

In [75]:
Q = 23740629843760
n_splits = 50
a_e = encrypt(a, n_splits, Q)
b_e = encrypt(b, n_splits, Q)
s_e = add(a_e, b_e)
decrypt(s_e, Q)

1107

### Course solution

In [78]:
import random

In [80]:
Q = 23740629843760239486723
x = 5

In [81]:
def encrypt(x, n_shares=3):
    n_shares = 3
    shares = list()
    for i in range(n_shares - 1):
        shares.append(random.randint(0, Q))

    shares.append(Q - (sum(shares) % Q) + x)
    return tuple(shares)

In [82]:
def decrypt(shares):
    return sum(shares) % Q

In [84]:
def add(a, b):
    return (s1 + s2 for s1, s2 in zip(a, b))

In [85]:
a = 342
b = 765
a + b

1107

In [86]:
decrypt(add(encrypt(a), encrypt(b)))

1107

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [25]:
BASE=10
PRECISION=4

In [26]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [27]:
encode(3.5)

35000

In [28]:
decode(35000)

3.5

In [29]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [30]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [31]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [32]:
x = x.share(bob, alice, secure_worker)

In [33]:
bob._objects

{35498656553: tensor([  10235770278698899, 1401398179551373756, 2277280072169145491,
          636965538565031298,  913795591610271305])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [34]:
y = x + x

In [35]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:23637986557 -> bob:30254176063]
	-> (Wrapper)>[PointerTensor | me:18229131498 -> alice:75856222543]
	-> (Wrapper)>[PointerTensor | me:34301722959 -> secure_worker:75419815101]
	*crypto provider: me*

In [36]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [37]:
x = th.tensor([0.1,0.2,0.3])

In [38]:
x

tensor([0.1000, 0.2000, 0.3000])

In [39]:
x = x.fix_prec()

In [40]:
x.child.child

tensor([100, 200, 300])

In [41]:
y = x + x

In [42]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [43]:
x = th.tensor([0.1, 0.2, 0.3])

In [44]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [45]:
y = x + x

In [46]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation